# Assignment 2

In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot

data = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")  ##dataframe is a datatype
df = data

df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df.Date.dt.year
df['Month'] = df.Date.dt.month
df['Day'] = df.Date.dt.day
df['Hour'] = pd.to_datetime(df['Time'])
df['Hour'] = df.Hour.dt.hour

focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

## Part 1: Questions to text and lectures.

A) Please answer my questions to the Segal and Heer paper we read during lecture 7 and 8.

* What is the *Oxford English Dictionary's* defintion of a narrative?

> a description of events, especially in a novel; the act, process or skill of telling a story

* What is your favorite visualization among the examples in section 3? Explain why in a few words.

> My favorite sample is *The Minnesota Employment Explorer*:
> * It's friendly to users with Mouse-hover for details and animated transition.
> * It include social interaction features.

* What's the point of Figure 7?

> It describes different patterns of Design Space Dimensions: (1) genre, (2) visual narrative tactics, and (3) narrative structure
tactics.

* Use Figure 7 to find the most common design choice within each category for the Visual narrative and Narrative structure (the categories within visual narrative are 'visual structuring', 'highlighting', etc).

> The most common choice within each categories:
> * Genre: Annotated Graph / Map
> * Visual structure: Consistent Visual Platform
> * Highlighting: Feature Distinction
> * Transition Guidance: object Continuity
> * Ordering: User Directed Path
> * Interactivicity: Filtering / Selection / Search
> * Messaging: Captions / Headlines

* Check out Figure 8 and section 4.3. What is your favorite genre of narrative visualization? Why? What is your least favorite genre? Why?

> * My favorite genre is Flow chart. It contains the whole process and details about a project, easy to understand.
> * My least favorite genre is magazine. It's filled with words and hard to find the key points.

B) Also please answer the questions to my talk on [explanatory data visualization](https://www.youtube.com/watch?v=yHKYMGwefso)

* What are the three key elements to keep in mind when you design an explanatory visualization?

> * Start with a question
> * Allow exploration
> * Know your readers

* In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
  - Go online and find a visualization that follows these principles (don't use one from the video). 
  - Explain how it does achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.

> Example from https://data.sfgov.org/d/tmnf-yvry/visualization: a visualization for SF dataset.
> * overview: It can select different Dimension like category for Overview.
> * Zoom and filter: It can add some measures for filtering and display the timeline of different type of crimes.
> * Details on demand: you can hover your mouse on the chart and it will show the details. 

* Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

> * Explanatory data analysis don't give the consequences directly, it encourages readers to find the patterns behind the data by show different forms of data.
> * Exploratory data analysis try to present specific results to demonstrate some hypothesis.

## Part 2: Random forest and weather

The aim here is to recreate the work you did in Part 1-3 of the Week 7 lecture. I've phrased things differently relative to the exercise to make the purpose more clear. 

Part 2A: Random forest binary classification. 
* Using the and instructions and material from Week 7, build a *random forest* classifier to distinguish between two types (you choose) of crime using on spatio-temporal (where/when) features of data describing the two crimes. When you're done, you should be able to give the classifier a place and a time, and it should tell you which of the two  types of crime happened there.
  - Explain about your choices for training/test data, features, and encoding. (You decide how to present your results, but here are some example topics to consider: Did you balance the training data? What are the pros/cons of balancing? Do you think your model is overfitting? Did you choose to do cross-validation? Which specific features did you end up using? Why? Which features (if any) did you one-hot encode? Why ... or why not?))
  - Report accuracy. Discuss the model performance.
  
  
Part 2B: Info from weather features.
* Now add features from weather data to your random forest. 
  - Report accuracy. 
  - Discuss how the model performance changes relative to the version with no weather data.
  - Discuss what you have learned about crime from including weather data in your model.

### Part 2A
**crime type**: `BURGLARY or FRAUD`

**feature**: 

* `DayOfWeek` 
* `PD District` 
* Hour of the day (1-24)
* Month of the year (1-12)

In [3]:
df1 = df[(df["Category"] == 'BURGLARY') | (df["Category"] == 'FRAUD')]
df1['Category'].replace('BURGLARY',0,inplace = True)
df1['Category'].replace('FRAUD',1,inplace = True)

C:\Users\xh975_9h4qt8f\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [4]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier  

category = df1['Category']
day_dum = pd.get_dummies(df1['DayOfWeek'])
dist_dum = pd.get_dummies(df1['PdDistrict'])
hour_dum = pd.get_dummies(df1['Hour'])
month_dum = pd.get_dummies(df['Month'])

In [5]:
testData = pd.concat([category, day_dum, dist_dum, hour_dum, month_dum], axis=1,join='inner')
testData = testData.reset_index(drop=True)
testData.head()

,Category,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,...,3,4,5,6,7,8,9,10,11,12
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Labels are the values we want to predict
labels = np.array(testData['Category'])
# Remove the labels from the features
testData= testData.drop('Category', axis = 1)
# Saving feature names for later use
feature_list = list(testData.columns)
# Convert to numpy array
testData = np.array(testData)

In [7]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(testData, labels, test_size = 0.25, random_state = 42)

In [8]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (99311, 53)
Training Labels Shape: (99311,)
Testing Features Shape: (33104, 53)
Testing Labels Shape: (33104,)


In [10]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 50, random_state = 42, max_depth=20)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(max_depth=20, n_estimators=50, random_state=42)

In [11]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.38 degrees.


In [12]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: nan %.
<ipython-input-12-f46941e48193>:2: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_labels)
<ipython-input-12-f46941e48193>:2: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * (errors / test_labels)


## Part 3: Data visualization

* Create the Bokeh visualization from Part 2 of the Week 8 Lecture, displayed in a beautiful `.gif` below. 
* Provide nice comments for your code. Don't just use the `# inline comments`, but the full Notebook markdown capabilities and explain what you're doing.


1. **Choose Take the data for the period of 2010-2018 and group it by hour-of-the-day (see Week 2)**

In [16]:
df3=df[(df.Year>2009) & (df.Year<2019)]

In [17]:
num_df = pd.DataFrame(columns=focuscrimes)
Count = df3.groupby('Category').size()

In [19]:
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

hour = set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

2. **Normalization for each type of data.**

In [20]:
for x in focuscrimes:
    series = df3[df3["Category"] == x].groupby('Hour').size()
    for y in hour:
        count_hour = series[y]
        count_total = Count[x]
        num_df.loc[y,x] = count_hour / count_total

In [21]:
num_df.head()

,TRESPASS,DISORDERLY CONDUCT,ROBBERY,BURGLARY,DRUG/NARCOTIC,LARCENY/THEFT,WEAPON LAWS,PROSTITUTION,VEHICLE THEFT,VANDALISM,STOLEN PROPERTY,DRIVING UNDER THE INFLUENCE,ASSAULT,DRUNKENNESS
0,0.028392,0.0516698,0.0553368,0.040437,0.0346767,0.0394609,0.0546579,0.129965,0.035958,0.0544846,0.0444687,0.120321,0.0564271,0.0807255
1,0.020475,0.0380172,0.0600345,0.0279466,0.0206792,0.025276,0.0394889,0.0961672,0.0242048,0.0381853,0.0335561,0.112002,0.0480985,0.0769384
2,0.025571,0.0323461,0.0603454,0.0316854,0.0167675,0.0154519,0.0339654,0.0609756,0.0182584,0.0355455,0.0294639,0.0989305,0.043706,0.0703608
3,0.021294,0.0182735,0.0373748,0.0334816,0.0123906,0.00995488,0.0237428,0.0365854,0.0119927,0.0259071,0.023462,0.0475342,0.023417,0.0269085
4,0.016198,0.0151229,0.0234888,0.0298891,0.00919778,0.0065595,0.0162407,0.0195122,0.010077,0.0177113,0.0200518,0.0172311,0.0145655,0.0141519


3. **convert Pandas Dataframe to Bokeh ColumnDataSource**

In [22]:
# convert df to bokeh
src = ColumnDataSource(num_df)

In [26]:
hours = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
p = figure(x_range = FactorRange(factors=hours),plot_height=250, toolbar_location=None, title="Crime Counts")

In [27]:
bar ={} 

focuscrime = ['DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'WEAPON LAWS', 'TRESPASS', 'STOLEN PROPERTY', 'DRUG/NARCOTIC', 'DISORDERLY CONDUCT', 'BURGLARY', 'DRUNKENNESS', 'VANDALISM', 'LARCENY/THEFT', 'VEHICLE THEFT', 'PROSTITUTION', 'ASSAULT']
for indx,i in enumerate(focuscrime):
    bar[i] = p.vbar(x='hours', top=i, source= src, legend_label=i) 

In [28]:
p.legend.click_policy="mute" 
show(p) #displays your plot

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "hours" [renderer: GlyphRenderer(id=1038, glyph=VBar(id='1036', ...), ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "hours" [renderer: GlyphRenderer(id=1053, glyph=VBar(id='1051', ...), ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "hours" [renderer: GlyphRenderer(id=1067, glyph=VBar(id='1065', ...), ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a missp